<a href="https://colab.research.google.com/github/jeawonlll/KOHI2022_tutorials/blob/main/20220820/%5B2%5DKOHI_Imputation_Toy_Example_220820.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multivariate Imputation by Chained Equations (MICE) 및 kNN Imputation 실습

## 순서


*   Univariate feature imputation
*   Multivariate feature imputation
*   Nearest neighbors imputation





## Univariate feature imputation

### sklearn.impute.SimpleImputer


*   missing_values: int, float, str, np.nan 등 missing value로 지정할 수 있는 데이터 타입
*   strategy: imputation 전략으로 각각 'mean', 'median', 'most_fequent', 'constant'가 설정되어 있음
*   fill_value: strategy가 'constant'로 되어있는 경우 채워질 수 있는 값
*   copy: copy가 true로 되어있는 경우 imputation이 완료된 input X가 새롭게 생성이 되어지며, false로 되어있는 경우 input X 자체에 imputation이 적용되어짐





In [1]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer

toy_train_data = [[1, 2.3], [np.nan, 3], [7, 6], [8.4, 3], [2, np.nan]]
toy_test_data = [[np.nan, 4], [np.nan, np.nan], [5.5, 8], [np.nan, 9], [2.7, np.nan]]
simple_imp = SimpleImputer(missing_values=np.nan, strategy='mean')
simple_imp.fit(toy_train_data)

print(simple_imp.transform(toy_test_data))

[[4.6   4.   ]
 [4.6   3.575]
 [5.5   8.   ]
 [4.6   9.   ]
 [2.7   3.575]]


### 실습: float type에 대한 'median' imputation 

In [2]:
toy_train_data = [[1, 2.3], [np.nan, 3], [7.5, 6], [8.4, 3], [2.1, np.nan]]
toy_test_data = [[np.nan, 4.2], [np.nan, np.nan], [5.5, 8], [np.nan, 9.1], [2.7, np.nan]]
toy_dat = [[np.nan, np.nan],[3,5.4]]
simple_imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
simple_imp_mean.fit(toy_train_data)

print('### Imputation with Mean')
print(simple_imp_mean.transform(toy_test_data))
print('\n\n')

simple_imp_median = SimpleImputer(missing_values=np.nan, strategy="median")
simple_imp_median.fit(toy_train_data)

print('### Imputation with Median')
print(simple_imp_median.transform(toy_test_data))

### Imputation with Mean
[[4.75  4.2  ]
 [4.75  3.575]
 [5.5   8.   ]
 [4.75  9.1  ]
 [2.7   3.575]]



### Imputation with Median
[[4.8 4.2]
 [4.8 3. ]
 [5.5 8. ]
 [4.8 9.1]
 [2.7 3. ]]


### sklearn.impute.IterativeImputer



* estimator: 매 round-robin imputation step 마다 사용될 모델로 default는 Bayesian Ridge 모델로 되어 있음. 추가적으로 사용해 볼 수 있는 모델은 Decision Tree Regressor, Extra Trees Regressor 등이 있음.
* missing_values: 상기 작성된 내용과 동일
* max_iter: Imputation round의 maximum step 수
* tol: stopping condition에 해당하는 수치
* n_nearest_features: missing value를 estimate 하기 위해 사용할 '다른' feature의 개수
* initial_strategy: SimpleImputer의 'strategy' hyperparameter와 동일
* imputation_order: Feature 별로 imputation이 되는 순서. 가능한 값은 'ascending', 'descending', 'roman', 'arabic', 'random'이 있음.
* skip_complete: True로 세팅되어 있는 경우, 'fit' 단계에서 missing value가 전혀 없었던 feature를 'transform'할 때 initial imputation 만을 실시. (missing value가 많지 않은 데이터의 경우 해당 값을 True로 세팅하여 'fit'과 'transform'에 시간 단축을 기대해 볼 수 있음) 
* min_value: imputation 되는 값의 최소 기대값 (imputation 되는 값은 이 수치 아래로 떨어질 수 없음)
* max_value: imputation 되는 값의 최대 기대값 (imputation 되는 값은 이 수치 위로 올라갈 수 없음)

In [3]:
iter_imp = IterativeImputer(max_iter=10, random_state=0)
iter_imp.fit(toy_train_data)

print(iter_imp.transform(toy_test_data))

[[4.93535118 4.2       ]
 [4.70704617 3.51527006]
 [5.5        8.        ]
 [6.56917587 9.1       ]
 [2.7        3.33133661]]


In [4]:
iter_imp = IterativeImputer(max_iter=10, random_state=0,imputation_order="random")
iter_imp.fit(toy_train_data)

print(iter_imp.transform(toy_test_data))

[[4.93535118 4.2       ]
 [4.70704617 3.51529096]
 [5.5        8.        ]
 [6.56917587 9.1       ]
 [2.7        3.3314115 ]]


### 실습: max value 7, min value 3을 세팅

In [5]:
##Max Value: 7
##Min Value: 3
iter_imp = IterativeImputer(max_iter=10, random_state=0, max_value=7, min_value=3)
iter_imp.fit(toy_train_data)

print(iter_imp.transform(toy_test_data))

[[4.93535118 4.2       ]
 [4.70704617 3.51527006]
 [5.5        8.        ]
 [6.56917587 9.1       ]
 [2.7        3.33133661]]


### sklearn.impute.KNNImputer

* missing_values: 상기 내용과 동일
* n_neighbors: kNN 모델의 neighbor k의 수
* weights: neighbor의 영향력 계산 방식. 'uniform'의 경우 모든 neighbor의 영향력은 동일하며 'distance'로 세팅할 경우 가까이 있는 neighbor의 경우 더 큰 영향을 주게 됨.
* metric: distance를 계산하는 metric 방법


In [6]:
knn_imp = KNNImputer(n_neighbors=2, weights='uniform')
knn_imp.fit(toy_train_data)

print(knn_imp.transform(toy_test_data))

[[7.95  4.2  ]
 [4.75  3.575]
 [5.5   8.   ]
 [7.95  9.1  ]
 [2.7   4.15 ]]


### 실습: n_neigbhors를 3개로 설정, weights은 distance 방법을 사용하여 KNNImputer fiting 실시 후 test toy 데이터에 대한 imputation 실시

In [7]:
##n_neigbhors: 3
##weights: distance
knn_imp = KNNImputer(n_neighbors=3, weights='distance')
knn_imp.fit(toy_train_data)

print(knn_imp.transform(toy_test_data))

[[6.10534351 4.2       ]
 [4.75       3.575     ]
 [5.5        8.        ]
 [6.22415613 9.1       ]
 [2.7        3.21937624]]
